# Building A Personal Safety Equipment Detection System

### Loading all the imports

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import tensorflow as tf
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

**Dataset Details:- Hardhat/head detection dataset, annotation format: Pascal VOC (XMLs)**
- **Dataset Size:- `Data with Annotations: 4750` and `Data without Annotations (Test): 250`**
- **Dataset classes of interest: `helmet` & `head`**

### Transforming Pascal VOC Annotations into YOLO Format

In [5]:
image_dir = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\images"
annotation_dir = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\annotations"
yolo_dir = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\labels"
os.makedirs(yolo_dir, exist_ok=True)

classes = {"helmet":0, "head":1}

for xml_file in glob.glob(os.path.join(annotation_dir, "*.xml")):
    image_file = os.path.join(image_dir, os.path.basename(xml_file).replace(".xml",".png"))
    if os.path.exists(image_file):
        img_read = cv2.imread(image_file)
        img_width, img_height, channels = img_read.shape
        
        # parse the xml file
        tree = ET.parse(xml_file)
        root = tree.getroot()
    
        # get the txt_filename (without extension)
        txt_filename = os.path.splitext(os.path.basename(xml_file))[0] + ".txt"
        
        # open a new yolo label file
        with open(os.path.join(yolo_dir,txt_filename), "w") as f:
            for obj in root.findall("object"):
                class_name = obj.find("name").text
                # skip classes not in mapping
                if class_name not in classes:
                    continue
                class_id = classes[class_name]
                bbox = obj.find("bndbox")
                xmin = int(bbox.find("xmin").text)
                ymin = int(bbox.find("ymin").text)
                xmax = int(bbox.find("xmax").text)
                ymax = int(bbox.find("ymax").text)
                # converting coordinates to YOLO format
                x_center = (xmin + xmax) / 2.0 / img_width
                y_center = (ymin + ymax) / 2.0 / img_height
                width = (xmax - xmin) / img_width
                height = (ymax - ymin) / img_height
                
                # Write the converted annotation to file
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
            
print("Conversion completed! YOLO labels saved in:", yolo_dir)

Conversion completed! YOLO labels saved in: C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\labels


### Visualizing YOLO Dataset

In [8]:
image_dir = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\images"
annotation_dir = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\annotations"
visualize_dir = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\vis"
os.makedirs(visualize_dir, exist_ok=True)

class_names = ["helmet", "head"]
object_colors = [(0,0,255), (0,255,0)]
hist = [0, 0]

def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize the bounding box coordinates
    out_img = image.copy()
    img_width, img_height, channels = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # denormalize the coordinates
        xmin = int(x1*img_width)
        ymin = int(y1*img_height)
        xmax = int(x2*img_width)
        ymax = int(y2*img_height)
        width = xmax - xmin
        height = ymax - ymin
        c = int(labels[box_num])
        cv2.rectangle(out_img, (xmin, ymin), (xmax, ymax), color=object_colors[c], thickness=2)
        hist[c] = hist[c] + 1
        '''
        cv2.putText(image, class_names[int(labels[box_num])], (xmin+1, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        '''
    return out_img
# '*.[pj][np]g' it allows finding both .jpg, .jpeg, and .png files in one pattern.
for image_file in glob.glob(os.path.join(image_dir,'*.[pj][np]g')):
    label_file = image_file.replace("images","labels").replace(".jpg",".txt") # we can remove this
    label_file = label_file.replace(".png",".txt")
    
    print(image_file)
    image = cv2.imread(image_file)

    with open(label_file, 'r') as f:
        bboxes = []
        labels = []
        label_lines = f.readlines()
        for label_line in label_lines:
            label = label_line[0]
            bbox_string = label_line[2:]
            x_c, y_c, w, h = bbox_string.split(' ')
            x_c = float(x_c)
            y_c = float(y_c)
            w = float(w)
            h = float(h)
            bboxes.append([x_c, y_c, w, h])
            labels.append(label)
    result_image = plot_box(image, bboxes, labels)

    out_file = os.path.join(visualize_dir,os.path.basename(image_file).replace('.jpg', '.png'))
    #cv2.imwrite(out_file, cv2.hconcat([image, result_image]))
    cv2.imwrite(out_file, result_image)
    print(hist)

C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers0.png
[7, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers1.png
[16, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers10.png
[19, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers100.png
[24, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers1000.png
[26, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers1001.png
[30, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers1002.png
[34, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers1003.png
[40, 6]
C:\Users\kbtha\Desktop\PSE Detection System\Dataset\HardHat_Dataset\images\hard_hat_workers1004.png
[45, 6]
C:\Users\kbtha\Desktop\PSE Detection S

### Organize Images and Labels in YOLO Structure

In [10]:
import os
import shutil
import random

# Define paths
dataset_path = r"C:\Users\kbtha\Desktop\PSE Detection System\data"
os.makedirs(dataset_path, exist_ok=True)
image_folder = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\images"
label_folder = "C:\\Users\\kbtha\\Desktop\\PSE Detection System\\Dataset\\HardHat_Dataset\\labels"

# Create YOLO directories
for folder in ["train/images", "train/labels", "val/images", "val/labels"]:
    os.makedirs(os.path.join(dataset_path, folder), exist_ok=True)

# Get all image files (supports .jpg, .png)
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png"))]

# Shuffle and split into train/val (80% train, 20% val)
random.shuffle(image_files)
split_index = int(0.95 * len(image_files))
train_images = image_files[:split_index]
val_images = image_files[split_index:]

# Function to move files to respective folders
def move_files(file_list, split_type):
    for file in file_list:
        image_src = os.path.join(image_folder, file)
        label_src = os.path.join(label_folder, file.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        # Destination paths
        image_dst = os.path.join(dataset_path, f"{split_type}/images", file)
        label_dst = os.path.join(dataset_path, f"{split_type}/labels", file.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        # Move image
        shutil.copy(image_src, image_dst)
        
        # Move corresponding label if exists
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)

# Move training and validation files
move_files(train_images, "train")
move_files(val_images, "val")

print("Dataset structured in YOLO format successfully!")

Dataset structured in YOLO format successfully!


### Generate data.yaml Automatically

In [19]:
# Define class names
class_names = ["helmet", "head"]

# Create `data.yaml`
yaml_content = rf"""train: {dataset_path}\train\images\
val: {dataset_path}\val\images\

nc: {len(class_names)}
names: {class_names}
"""

# Save `data.yaml`
with open(os.path.join(dataset_path, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("data.yaml file created successfully!")

data.yaml file created successfully!


### YOLO Dataset Statistics: Count Images and Labels

In [22]:
import glob
train_img_path = r"C:\Users\kbtha\Desktop\PSE Detection System\data\train\images"
val_img_path = r"C:\Users\kbtha\Desktop\PSE Detection System\data\val\images"

def get_stats(train=True):
    if train:
        mask = os.path.join(train_img_path,'*.[pj][np]g')
    else:
        mask = os.path.join(val_img_path,'*.[pj][np]g')
        
    class_names = ["helmet", "head"]
    hist = [0,0]
    num_images = 0
    for image_file in glob.glob(mask):
        label_file = image_file.replace("images","labels").replace(".jpg",".txt")
        label_file = label_file.replace(".png",".txt")
        num_images = num_images + 1
        
        with open(label_file, 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = int(label_line[0])
                hist[label] = hist[label]+1
                
    print("Number of images: ", num_images)
    for c in enumerate(class_names):
        print(c[1], ":", hist[c[0]])

print("Training Set Stats\n")
get_stats(True)
print("Validation Set Stats\n")
get_stats(False)

Training Set Stats

Number of images:  4512
helmet : 17117
head : 5217
Validation Set Stats

Number of images:  238
helmet : 923
head : 274
